# Objetivo:
- Plotar ocorrências de focos de calor fornecidos pelo INPE num mapa interativo usando o [Leafmap](https://leafmap.org/)
- O `leafmap` é um pacote relativamente novo, versátil e amigável para mapeamento interativo de dados em Python. Desenvolvido por Qiusheng Wu como um derivado do pacote `geemap`, usado exclusivamente no contexto do **Google Earth Engine**, o `leafmap` oferece uma sintaxe simples que permite a criação de mapas interativos de forma eficiente através da integração com outros pacotes como `Folium` e `ipyleaflet`. Antes de mais nada, vamos importá-lo:

# Acesso aos dados:
- Focos de calor: https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/
- Geojson dos municípios do Brasil: https://github.com/tbrugz/geodata-br
- Geojson do contorno de cada estado num arquivo: https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson
- Geojson do contorno de cada estado em arquivos separados: https://github.com/giuliano-macedo/geodata-br-states/blob/main/geojson/br_states/

# Carregando os dados

In [3]:
%%time
# Importa bibliotecas
from datetime import datetime, timedelta, date
import datetime
import pandas as pd

# URL dos dados do INPE
url_diario = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/Brasil/'

# Loop nos dias
df = pd.DataFrame()
anos, meses, dias = [], [], []
for i in range(8):

    # gera as datas
    data = datetime.datetime.now() - timedelta(days=i)
    ano, mes, dia = date.strftime(data,'%Y'), date.strftime(data,'%m'), date.strftime(data,'%d')
    anos.append(ano)
    meses.append(mes)
    dias.append(dia)

    # leitura dos dados
    df0 = pd.read_csv(f'{url_diario}focos_diario_br_{ano}{mes}{dia}.csv')

    # junta a tabela que foi lida com a anterior
    df = pd.concat([df, df0], ignore_index=True)

# seleciona os focos de Minas Gerais
df_mg = df[df['estado']=='MINAS GERAIS']

# mostra os dados
df_mg 

CPU times: total: 328 ms
Wall time: 4.05 s


,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp
67,b0cd5836-c330-30de-a094-c8ab96ba1b2f,-19.958599,-48.358601,2024-07-06 00:48:24,METOP-B,CONCEIÇÃO DAS ALAGOAS,MINAS GERAIS,Brasil,3117306,31,33,NaN,NaN,NaN,Cerrado,NaN
68,4b167c6d-3549-3d28-a2bc-4d9f296077a7,-19.116400,-46.546398,2024-07-06 00:48:31,METOP-B,SERRA DO SALITRE,MINAS GERAIS,Brasil,3166808,31,33,NaN,NaN,NaN,Cerrado,NaN
69,a854c9f6-8e2b-3fbe-87c6-5ff8c94e4685,-19.703600,-49.269600,2024-07-06 00:48:31,METOP-B,ITAPAGIPE,MINAS GERAIS,Brasil,3133402,31,33,NaN,NaN,NaN,Cerrado,NaN
70,e484e182-9720-3225-91f5-2380c2b4029b,-19.118200,-46.554298,2024-07-06 00:48:31,METOP-B,SERRA DO SALITRE,MINAS GERAIS,Brasil,3166808,31,33,NaN,NaN,NaN,Cerrado,NaN
71,33267962-e558-311b-abf5-925475943747,-19.237600,-48.686699,2024-07-06 00:48:37,METOP-B,PRATA,MINAS GERAIS,Brasil,3152808,31,33,NaN,NaN,NaN,Cerrado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66210,8fd42749-ae52-353b-85e3-120031d22d2f,-15.060700,-43.865200,2024-06-29 20:16:30,GOES-16,MATIAS CARDOSO,MINAS GERAIS,Brasil,3140852,31,33,NaN,NaN,NaN,Caatinga,83.1
66264,0c473ed8-53bb-3544-83b2-89a4cbd0940e,-15.060100,-43.889100,2024-06-29 20:26:30,GOES-16,MATIAS CARDOSO,MINAS GERAIS,Brasil,3140852,31,33,NaN,NaN,NaN,Cerrado,98.6
66265,314f0209-f869-35fb-8667-af5257549ad5,-15.060700,-43.865200,2024-06-29 20:26:30,GOES-16,MATIAS CARDOSO,MINAS GERAIS,Brasil,3140852,31,33,NaN,NaN,NaN,Caatinga,142.1
66386,3f223308-5673-3a81-9be5-d5ff2af51897,-20.856300,-45.523500,2024-06-29 22:17:01,GOES-16,CRISTAIS,MINAS GERAIS,Brasil,3120201,31,33,NaN,NaN,NaN,Cerrado,123.3


# Mapa: em forma de `Pontos`

In [4]:
# importa biblioteca
import leafmap.leafmap as leafmap

# configuração do mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_xy_data(df_mg, x="lon", y="lat")

# adiciona contorno dos municípios do Brasil no mapa
shp_mg = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json'
Map.add_geojson(shp_mg, layer_name="Munícipios de MG")

# exibe o mapa
Map

Map(center=[-15, -60], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [5]:
# salva imagem para HTML
Map.to_html("map_pontos.html")

# Mapa: em forma de `Markers`

In [6]:
# importa biblioteca
import leafmap.leafmap as leafmap

# configuração do mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_circle_markers_from_xy(df_mg,
                               x="lon",
                               y="lat", 
                               radius=2, 
                               color="red",
                               fill=True,
                               fill_color="red",
                               fill_opacity=1)

# adiciona contorno dos estados
shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados do Brasil")

# exibe o mapa
Map

Map(center=[-15, -60], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [7]:
# salva imagem para HTML
Map.to_html("map_Markers.html")

# Mapa: em forma de `Heatmap`

## Leitura do dado anual do INPE

In [8]:
%%time
# Importa bibliotecas
from datetime import datetime, timedelta, date
import datetime
import pandas as pd

# URL dos dados mensais do INPE
url_mensal = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/Brasil_sat_ref/'

# nome do arquivo
filename = 'focos_br_ref_2020.zip'

# leitura dos dados
df = pd.read_csv(f'{url_mensal}{filename}', compression='zip', usecols=['lat','lon','data_pas'])

# renomeia algumas colunas
df.rename(columns={'lat': 'latitude', 'lon': 'longitude'}, inplace=True)

# cria uma coluna como valor um para cada ocorrência de foco de calor
df['count'] = 1

# mostra os dados
df

CPU times: total: 578 ms
Wall time: 1.98 s


,latitude,longitude,data_pas,count
0,-3.650,-54.732,2020-06-16 17:00:00,1
1,-4.128,-47.074,2020-06-16 17:00:00,1
2,-10.513,-53.483,2020-06-16 16:55:00,1
3,-10.504,-53.498,2020-06-16 16:55:00,1
4,-10.494,-46.488,2020-06-16 16:55:00,1
...,...,...,...,...
222792,-23.163,-49.164,2020-10-02 17:15:00,1
222793,-9.558,-66.777,2020-10-13 18:40:00,1
222794,-21.691,-44.016,2020-10-13 17:00:00,1
222795,-16.451,-46.062,2020-10-13 17:00:00,1


## Plota mapa

In [9]:
# importa biblioteca
import leafmap.leafmap as leafmap

# configuração do mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_heatmap(df,
                x="longitude",
                y="latitude",
                value="count",
                name="Heat map",
                radius=20)

# adiciona contorno dos estados
shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados do Brasil")

# exibe o mapa
Map

Map(center=[-15, -60], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [10]:
# salva imagem para HTML
Map.to_html("map_heatmap.html")

# Mapa: para os últimos 7 dias da semana

## Processando os dados

In [26]:
%%time
# Importa bibliotecas
from datetime import datetime, timedelta, date
import datetime
import pandas as pd

# URL dos dados do INPE
url_diario = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/Brasil/'

# Loop nos dias
df = pd.DataFrame()
anos, meses, dias, ano_mes_dia = [], [], [], []
for i in range(8):

    # gera as datas
    data = datetime.datetime.now() - timedelta(days=i)
    
    ano, mes, dia = date.strftime(data,'%Y'), date.strftime(data,'%m'), date.strftime(data,'%d')
    ano_mes_dia.append(f'{ano}-{mes}-{dia}')     
    anos.append(ano)
    meses.append(mes)
    dias.append(dia)

    # leitura dos dados
    df0 = pd.read_csv(f'{url_diario}focos_diario_br_{ano}{mes}{dia}.csv', usecols=['data_hora_gmt','lat','lon','satelite','municipio','estado'])

    # junta a tabela que foi lida com a anterior
    df = pd.concat([df, df0], ignore_index=True)

# ordena as datas
ano_mes_dia=sorted(ano_mes_dia)

# seleciona os focos de Minas Gerais e do Satélite AQUA
df_mg = df[ (df['estado']=='MINAS GERAIS') & (df['satelite']=='AQUA_M-T') ]

# transforma a data em índice
df_mg['data_hora_gmt'] = pd.to_datetime(df_mg['data_hora_gmt']) # transformar a coluna 'acq_date' em formato datetime
#df_mg.set_index('data_hora_gmt', inplace=True) # definir a coluna 'acq_date' como índice do DataFrame

# ordena o dataframe
#df_mg.sort_index(axis=0, inplace=True)
df_mg.sort_values(by=['data_hora_gmt'], inplace=True)

# cria a coluna com o ano, mês e dia
df_mg['Data'] = df_mg['data_hora_gmt'].apply(lambda x: x.strftime('%Y-%m-%d'))

# elimina algumas colunas
df_mg.drop(columns=['data_hora_gmt'], inplace=True)

# renomeia os nomes das colunas
df_mg.rename(columns={'lat': 'Latitude', 'lon': 'Longitude', 'satelite': 'Satélite', 'municipio': 'Município','estado': 'Estado'}, inplace=True)

# salva arquivo CSV
df_mg.to_csv('focos_diario.csv', index=False)

# salva arquivo CSV para cada dia
for dia in ano_mes_dia: df_mg[ df_mg['Data'] == dia].to_csv(f'focos_diario_{dia}.csv', index=False)

# mostra os dados
df_mg

CPU times: total: 391 ms
Wall time: 3.88 s


,Latitude,Longitude,Satélite,Município,Estado,Data
62356,-19.65274,-41.24707,AQUA_M-T,AIMORÉS,MINAS GERAIS,2024-06-29
62339,-19.16050,-43.59877,AQUA_M-T,SANTANA DO RIACHO,MINAS GERAIS,2024-06-29
62340,-17.91201,-42.55101,AQUA_M-T,ARICANDUVA,MINAS GERAIS,2024-06-29
62351,-21.26426,-42.90792,AQUA_M-T,ASTOLFO DUTRA,MINAS GERAIS,2024-06-29
62348,-20.57470,-44.86234,AQUA_M-T,CARMO DA MATA,MINAS GERAIS,2024-06-29
...,...,...,...,...,...,...
4411,-21.29019,-43.02300,AQUA_M-T,PIRAÚBA,MINAS GERAIS,2024-07-06
4417,-15.71770,-41.02919,AQUA_M-T,DIVISÓPOLIS,MINAS GERAIS,2024-07-06
4416,-15.66480,-41.61570,AQUA_M-T,ÁGUAS VERMELHAS,MINAS GERAIS,2024-07-06
6603,-20.42378,-46.64646,AQUA_M-T,DELFINÓPOLIS,MINAS GERAIS,2024-07-06


## Plota mapa - todos focos no mesmo mapa ao mesmo tempo
- The list of available icon names can be found at https://fontawesome.com/v4/icons.

In [12]:
# importa biblioteca
import leafmap

# configuração do mapa
Map = leafmap.Map(center=(-18, -40), zoom=6.3, layers_control=True)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota mapa
Map.add_points_from_xy('focos_diario.csv',
                       x="Longitude",
                       y="Latitude",
                       color_column="Data",                       
                       marker_colors=['lightgray', 'gray', 'blue', 'darkblue', 'green', 'darkgreen', 'red','darkred'],
                       icon_colors=['lightgray', 'gray', 'blue', 'darkblue', 'green', 'darkgreen', 'red','darkred'],
                       spin=False,
                       add_legend=True,
                       layer_name='Focos de Calor Diário')

# adiciona contorno dos municípios do Brasil no mapa
shp_mg = 'https://github.com/giuliano-macedo/geodata-br-states/blob/main/geojson/br_states/br_mg.json'
Map.add_geojson(shp_mg, layer_name="Munícipios de MG")

# exibe o mapa
Map

Map(center=[-18, -40], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Plota mapa - todos focos no mesmo mapa, podendo selecionar o dia separado

In [34]:
# importa biblioteca
import leafmap

# configuração do mapa
Map = leafmap.Map(center=(-18, -40), zoom=6.3, layers_control=True)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# define as cores de cada dia
cores = ['lightgray', 'gray', 'blue', 'darkblue', 'green', 'darkgreen', 'red', 'darkred']

# plota mapa
for i, dia in enumerate(ano_mes_dia):
    qte_focos = df=pd.read_csv(f'focos_diario_{dia}.csv').shape[0]
    Map.add_points_from_xy(f'focos_diario_{dia}.csv',
                           x="Longitude",
                           y="Latitude",
                           color_column="Data",                       
                           marker_colors=[cores[i]],
                           spin=False,
                           add_legend=False,
                           layer_name=f'{dia} = {qte_focos}')

# adiciona contorno dos municípios do Brasil no mapa
shp_mg = 'https://github.com/giuliano-macedo/geodata-br-states/blob/main/geojson/br_states/br_mg.json'
Map.add_geojson(shp_mg, layer_name="MG")

# exibe o mapa
Map

Map(center=[-18, -40], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…